In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
import sys

In [9]:
sys.path.append("/Users/mohammadshoaib/Codes/tensorflow-test/GSOC-22-Video-Swin-Transformers")

In [10]:
from VideoSwinTransformer import model_configs, I3DHead_tf , CosineAnnealingWithWarmupSchedule

In [11]:
def get_model(num_classes, backbone ,shape_of_input=(3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    
    x = backbone(inputs, training= True)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [12]:
# loading the tf2 model with converted pre-trained weights
path_to_model_weights = "/Users/mohammadshoaib/Codes/tensorflow-test/GSOC-22-Video-Swin-Transformers/swin_tiny_patch244_window877_kinetics400_1k_tf"
backbone = tf.keras.models.load_model(path_to_model_weights)

# uncomment the following line if you want to initialize the backbone model with random values
# backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)

2022-10-05 17:28:25.814521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-05 17:28:25.815213: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [39]:
batch_size = 1
num_frames = 32
resolution = 224
channels = 3
sample_count = 10
num_classes = 5

shape_of_input = (sample_count, 3, num_frames, resolution,resolution)

In [41]:
videos = tf.random.normal(shape_of_input) #for _ in range(sample_count)
labels = tf.random.uniform(shape=[sample_count], minval=0, maxval=num_classes, dtype='int64')
labels = tf.one_hot(labels, num_classes)

len(videos), videos[0].shape, labels.shape

(10, TensorShape([3, 32, 224, 224]), TensorShape([10, 5]))

In [43]:
num_epochs = 3

warmup_epoch = 2
total_steps = int(num_epochs * sample_count / batch_size)

# Compute the number of warmup batches.
warmup_steps = int(warmup_epoch * sample_count / batch_size)

train_steps = sample_count// batch_size
total_train_steps = train_steps * num_epochs
test_steps = sample_count // batch_size

In [44]:
model = get_model(num_classes,  backbone,shape_of_input=shape_of_input)

## backbone model has a multiplier of 0.1 for learning rate. To implement that MultiOptimizer is being used
lr_backbone = CosineAnnealingWithWarmupSchedule(learning_rate_base=.001,
                                    total_steps=total_steps,
                                    warmup_learning_rate=0.0,
                                    warmup_steps=warmup_steps,
                                    hold_base_rate_steps=0)
lr_classifier = CosineAnnealingWithWarmupSchedule(learning_rate_base=.01,
                                        total_steps=total_steps,
                                        warmup_learning_rate=0.0,
                                        warmup_steps=warmup_steps,
                                        hold_base_rate_steps=0)

optimizer_backbone = tfa.optimizers.AdamW(weight_decay= 0.05,learning_rate=lr_backbone, beta_1= 0.9, beta_2=0.999, epsilon=1e-8)
optimizer_classifier = tfa.optimizers.AdamW(weight_decay= 0.05,learning_rate=lr_classifier, beta_1= 0.9, beta_2=0.999, epsilon=1e-8)

optimizers_and_layers = [(optimizer_backbone, model.layers[1]), (optimizer_classifier, model.layers[2])]
optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

loss_obj = tf.keras.losses.CategoricalCrossentropy(from_logits=True,label_smoothing=0.1,
                                    reduction=tf.keras.losses.Reduction.SUM)

metrics=["top_k_categorical_accuracy"] 

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

5 768 {'type': 'CrossEntropyLoss'} avg 0.5 0.01 {'training': True}


In [45]:
results = model.fit(videos,labels,
    epochs=num_epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=[],
    validation_freq=1,
    verbose=1)

Epoch 1/3


2022-10-05 17:35:08.276567: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


lr = 5e-05
lr = 0.0005
lr = 5e-05
lr = 0.0005


2022-10-05 17:35:15.123365: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 49s 3s/step - loss: 1.6442 - top_k_categorical_accuracy: 1.0000
Epoch 2/3
10/10 [==============================] - 31s 3s/step - loss: 1.6114 - top_k_categorical_accuracy: 1.0000
Epoch 3/3
10/10 [==============================] - 34s 3s/step - loss: 1.5692 - top_k_categorical_accuracy: 1.0000
